In [56]:
import time
import random
import requests
import pandas as pd
import time
import random
import yaml 
import json
import torch
import datetime as dt
import re

with open("shopee.yaml","rb") as f:
    cfg=yaml.load(f,Loader=yaml.SafeLoader)
    cfg['device']=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    convert_name = cfg['convert_id_to_name']
    print(f"### Loading config {cfg}")

### Loading config {'P001': './json/iphone15promax_Apr26.json', 'P002': './json/samsungs23ultra_Apr26.json', 'P003': './json/samsungs24ultra.json', 'convert_id_to_name': {'ShopeeID_1039322456': 'TCMOBEO', 'ShopeeID_182298008': 'Đại Phát shop', 'ShopeeID_45958676': 'FUN OFFICIAL.', 'ShopeeID_42787830': 'DiDongStore.com', 'ShopeeID_48406301': 'SHOPSAMSUNG.VN LINHPHUKIEN.VN', 'ShopeeID_1039678074': 'ShopeeID_1039678074', 'ShopeeID_1107128132': 'IIlumidzShop', 'ShopeeID_1165274448': 'DMobile-HCM', 'ShopeeID_424715': 'Điện Thoại Giá Kho Official', 'ShopeeID_349543642': 'Tamanmobile', 'ShopeeID_65589552': 'SAMSUNG OFFICIAL STORE', 'ShopeeID_603809751': 'Omi Mobile', 'ShopeeID_181774937': 'GalaxyShopVN', 'ShopeeID_37679543': 'Hnam Mobile', 'ShopeeID_811629533': 'CuBeoShop', 'ShopeeID_897072353': 'PREMIUM STORE ONLINE', 'ShopeeID_297083882': 'ShopeeID_297083882', 'ShopeeID_36280069': 'TechGearVN', 'ShopeeID_88201679': 'Apple Flagship Store', 'ShopeeID_288286284': 'ShopDunk Official Store', 'Sh

In [2]:
def get_info(data):
  data_list = [{'ItemID': item['item_basic']['itemid'],
                'ShopID': item['item_basic']['shopid'],
                'Name': item['item_basic']['name'],
                # 'stock':item['item_basic']['stock'],
                # 'ctime':item['item_basic']['ctime'],
                # 'sold':item['item_basic']['sold'],
                'Quantity':item['item_basic']['historical_sold'],
                # 'liked_count':item['item_basic']['liked_count'],
                'ReviewCount':item['item_basic']['item_rating']['rating_count'][0],
                'Price':item['item_basic']['price'],
                # 'options':item['item_basic']['tier_variations'][0]['options'],
                'StarRating':item['item_basic']['item_rating']['rating_star']} 
              for item in data['items']]
  df=pd.DataFrame(data_list,index=None)
  return df


# Filter Products for Samsung
def filter_products(data,filter1,filter2):
    
    # Remove Product's Name that not related (s24, fe, iphone 12)
    names_lower = data['Name'].str.lower()
    pattern = re.compile('|'.join(filter1))
    indices = ~names_lower.str.contains(pattern)
    data=data[indices]

    # Remove Product's Name that not contain related words (ex: iphone 15 pro max/ samsung s23 ultra)
    names_lower = data['Name'].str.lower()
    pattern = re.compile(r'(?=.*{})'.format(')(?=.*'.join(filter2)), flags=re.IGNORECASE)
    indices = names_lower.str.contains(pattern)
    data=data[indices]

    data = data[data['Quantity']!=0]
    data.reset_index(inplace=True)
    # Storage
    for idx,name in enumerate(data['Name']):
        name = name.lower()
        if '256gb' in name:
            data.loc[idx,'Storage'] = '256GB'
        elif '512gb' in name.lower():
            data.loc[idx,'Storage'] = '512GB'
        elif '1tb' in name:
            data.loc[idx,'Storage'] = '1TB'
        if (('256gb' in name) and ('512gb' in name)) or (('512gb' in name) and ('1tb' in name)) or (('256gb' in name) and('512gb' in name) and  ('1tb' in name)) or (('256gb' in name) and  ('1tb' in name)):
            data.loc[idx,'Storage'] = 'Multi Storage'

    return data


In [4]:

# for product in save_name[1:]:
def not_exist_product(product,new,year,month,day):
    date = ''
    old = pd.read_excel(f'../../Data/Shopee/{product}_Apr17.xlsx')
    # new = pd.read_csv(f'../../Data/Shopee/{product}_Apr07.csv')

    list_item  = [i for i in new['ItemID']]
    list_id = [id_product for id_product in old['ItemID'] if int(id_product) not in list_item]

    df_new = []
    for idx,row in old.iterrows():
        # print(row['ItemID'])
        if int(row['ItemID']) in list_id:
            df_new.append(row)

    df_not_exist_product =pd.DataFrame(df_new)

    if df_new ==[]:
        print('The number of not exist product is 0')
        return df_not_exist_product
    
    df_not_exist_product['CrawlDate'] = dt.datetime(year,month,day)
    df_not_exist_product['ShopID']=df_not_exist_product['ShopID'].replace(convert_name)
    return df_not_exist_product

In [57]:
products=['P001','P002']
save_name = ['iphone_15_pro_max_info','galaxy_s23_ultra_info']
# ['ProductID', 'PlatformID', 'ShopID', 'Price', 'Quantity', 'Reviews count', 'Star rating', 'Crawl date']
df = pd.DataFrame()
filter_1 = [['máy tính','plus','samsung','galaxy','ốp lưng','ốp','iphone 11','iphone 12','iphone 13','iphone 14','cũ','Likenew'],['Likenew','a54','s24','s22','z','fold','flip4','s21','note','flip5','fe','s24+','iphone','ốp','ốp lưng','cũ']]
filter_2 = [['iphone','pro','max'],['s23','ultra']]

for idx,file in enumerate(products):
    with open(cfg[file],'r',encoding='utf-8') as f:
        data=json.load(f)
        print(f'Saving data of {file}')
        df_temp = get_info(data)
        df_temp['PlatformID'] = 'PL03'
        df_temp['ProductID'] = file
        # df_temp['CrawlDate'] = dt.datetime.today()
        df_temp['CrawlDate'] = dt.datetime(2024,4,26)
        df = filter_products(df_temp,filter_1[idx],filter_2[idx])
        df['ShopID'] = 'ShopeeID_' + df['ShopID'].astype(str)
        df['ShopID']=df['ShopID'].replace(convert_name)

        # Not exist product 
        df_not_exist_product = not_exist_product(save_name[idx],df,2024,4,26)
        # df_not_exist_product['CrawlDate'] = dt.datetime(2024,3,30)
        # df_not_exist_product['ShopID']=df_not_exist_product['ShopID'].replace(convert_name)

        # Combine two dataframe
        df = pd.concat([df, df_not_exist_product], ignore_index=True)

        df.to_excel(f'../../Data/Shopee/{save_name[idx]}_Apr26.xlsx',index=False)


Saving data of P001
Saving data of P002


In [21]:
df_edit = pd.read_excel('../../Data/Shopee/galaxy_s23_ultra_info_Apr26.xlsx')
df_old = pd.read_excel('../../Data/Shopee/galaxy_s23_ultra_info_Apr17.xlsx')
# df_old['ItemID'][1].type()


17    256GB
Name: Storage, dtype: object

In [62]:
import pandas as pd
df_edit = pd.read_excel('../../Data/Shopee/galaxy_s23_ultra_info_Apr26.xlsx')
df_old = pd.read_excel('../../Data/Shopee/galaxy_s23_ultra_info_Apr17.xlsx')
# df_edit.columns
for idx,storage in enumerate(df_edit['Storage']):
    if isinstance(storage,str) and storage!='Multi Storage':
        continue
    else:
        storage = df_old[df_old['ItemID']==int(df_edit.loc[idx,'ItemID'])]['Storage']
        # print(storage)
        if not storage.empty:
            storage_value = storage.values[0]
            df_edit.loc[idx,'Storage'] = storage_value
        else:
            print(df_edit.loc[idx,'ItemID'])
            print("No matching 'ItemID' found in df_old for index", idx)
print(df_edit['Storage'].isna().sum())
df_edit.to_excel(f'../../Data/Shopee/galaxy_s23_ultra_info_Apr26.xlsx',index=False)


0


In [27]:
# import pandas as pd
# df_edit = pd.read_excel('../../Data/Shopee/galaxy_s23_ultra_info_Apr26.xlsx')
# # df_edit.columns
# for idx,storage in enumerate(df_edit['Storage']):
#     if isinstance(storage,nan):
#         print('yes')
#         df_edit.loc[idx,'ShopID'] = 'ShopeeID_' + str(shop)
# # df_edit.to_excel('../../Data/Shopee/galaxy_s23_ultra_info_Apr17.xlsx',index=False)

In [66]:
df_edit = pd.read_excel('../../Data/Preprocessed_data/Market.xlsx')
df_edit.to_csv('../../Data/Preprocessed_data/Market.csv',index=False)


In [65]:
df_edit = pd.read_csv('../../Data/Preprocessed_data/Market.csv')
df_edit.to_excel('../../Data/Preprocessed_data/Market.xlsx',index=False)


### Reviews

In [71]:
# MAIN
def get_reviews(shop_id,item_id,prodid):
    url = "https://shopee.com.my/api/v2/item/get_ratings"
    limit = 1
    offset = 0
    reviews = []
    
    session = requests.Session()
    session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})
    
    while True:
        params = {
            "itemid": item_id,
            "shopid": shop_id,
            "offset": offset,
            "limit": limit,
            "filter": "0",
            "flag": "0",
            "sort": "0",
            "append": "0",
            "before_bundle": "",
            "language": "en",
        }
    
        response = session.get(url, params=params).json()
        if response["error"]:
            print(f"Error: {response['error']}")
            break
        elif response["data"]["ratings"]:
            for rating in response["data"]["ratings"][:201]:
                model_name = rating.get('product_items')[0].get('model_name')
                rating['model_name'] = model_name
                reviews.append(rating)
                offset += limit
        else:
            break
        
    df = pd.DataFrame(reviews) 

    print(f"Retrieved {len(reviews)} reviews.")
    df = df[[ 'itemid','ctime','shopid', 'comment', 'rating_star']]
    df.rename(columns={"itemid": "ItemID",'shopid':'ShopID','ctime':'DateReview', 'comment':'Review','rating_star':'StarRating'},inplace=True)
    df['ProductID'] = prodid
    df['PlatformID'] = 'PL03'
    df['DateReview'] = pd.to_datetime(df['DateReview'],unit='s').dt.date

    # print(f'Crawl shop: {shop_id}, item{item_id} completed')
    # df.to_excel(f'Shopee_shopid{shop_id}.xlsx', index=False)
    return df


In [ ]:
import os
import pandas as pd
import requests
# data_rw = pd.DataFrame()
prod_name = ['galaxy_s23_ultra','iphone_15_pro_max']

for name in prod_name:
    path = f'../../Data/Shopee/{name}_info.csv'
    if os.path.isfile(path):
        shopee_prod_info = pd.read_csv(path)
    else:
        continue
    
    shopee_prod_info = shopee_prod_info[shopee_prod_info['ReviewCount']!=0]
    data_rw = pd.DataFrame()

    for itemid, shopid,prodid in zip(shopee_prod_info['ItemID'],shopee_prod_info['ShopID'],shopee_prod_info['ProductID']):
        df_rw = get_reviews(shopid,itemid,prodid)
        data_rw = pd.concat([data_rw,df_rw],ignore_index=True)
    data_rw.to_csv(f'../../Data/Shopee/{name}_review.csv',index=False)
    print(f'Crawled {name} reviews')

# Done

In [ ]:
# import os
# import csv
# def concat_csv_review(directory):
#     csv_files = [file for file in os.listdir(directory) if file.startswith("Shopee_shopid")]
#     if not csv_files:
#         return "no CSV files found."
    
#     dfs=[]

#     for file_name in csv_files:
#         file_path=os.path.join(directory,file_name)
#         dfs.append(pd.read_csv(file_path))

#     df_sum = pd.concat(dfs,ignore_index=True)

#     df_sum.drop(['orderid','cmtid','ctime','userid',,'like_count','model_name'],axis=1,inplace=True)
#     df_sum.to_csv()
#     return df_sum

In [ ]:
import requests
import pandas as pd

url = "https://shopee.com.my/api/v2/item/get_ratings"
shop_id = "308461157"
item_id = "23354704062"
limit = 1
offset = 0
reviews = []

session = requests.Session()
session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})



while True:
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "offset": offset,
        "limit": limit,
        "filter": "0",
        "flag": "0",
        "sort": "0",
        "append": "0",
        "before_bundle": "",
        "language": "en",
    }

    response = session.get(url, params=params).json()
    if response["error"]:
        print(f"Error: {response['error']}")
        break
    elif response["data"]["ratings"]:
        print(offset)
        print (response["data"]["ratings"])

        for rating in response["data"]["ratings"]:
            model_name = rating.get('product_items')[0].get('model_name')
            rating['model_name'] = model_name
            reviews.append(rating)
            offset += limit

            time.sleep(random.randint(2,5))  
    else:
        break

df = pd.DataFrame(reviews)

print(f"Retrieved {len(reviews)} reviews.")
# extract 'skin suitability' and 'absorption' from 'comment' column
df[['skin_suitability', 'absorption']] = df['comment'].str.extract('Skin Suitability:(.*?)\\nAbsorption:(.*?)\\n', expand=True)

df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'rating', 'userid', 'shopid', 'comment', 'rating_star', 'skin_suitability', 'absorption', 'model_name']]

print('success')

df.to_csv('ShopeeScrap1.csv', index=False)


In [ ]:
import requests
import pandas as pd
import time
import random

def get_reviews(shop_id,item_id):
    url = "https://shopee.com.my/api/v2/item/get_ratings"
    limit = 1
    offset = 0
    reviews = []
    
    session = requests.Session()
    session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "offset": offset,
        "limit": limit,
        "filter": "0",
        "flag": "0",
        "sort": "0",
        "append": "0",
        "before_bundle": "",
        "language": "en",
    }
    response = session.get(url, params=params).json()
    print(response)
    for rating in response["data"]["ratings"][:10]:
        model_name = rating.get('product_items')[0].get('model_name')
        rating['model_name'] = model_name
        reviews.append(rating)
        offset += limit

    df = pd.DataFrame(reviews)
    print(f"Retrieved {len(reviews)} reviews.")
    
    df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'userid', 'shopid', 'comment', 'rating_star','like_count',  'model_name']]
    df.to_excel('temp.xlsx', index=False)

    
shop_id = "288286284"
item_id = "21993222972"
get_reviews(shop_id,item_id)

{'error_msg': None, 'data': {'ratings': [{'orderid': 153680409319298, 'itemid': 21993222972, 'cmtid': 13284042161, 'ctime': 1700535964, 'rating': 1, 'userid': 330045429, 'shopid': 288286284, 'comment': 'Chất lượng sản phẩm:tốt\nĐúng với mô tả:tốt\nTính năng nổi bật:màu titan tự nhiên\n\nShopdunk vận chuyển nhanh chóng, rất tốt. Mua ở đây yên tâm hơn mua ở apple flagship store nhiều. Sẽ ủng hộ mua của shopdunk', 'rating_star': 5, 'status': 2, 'mtime': 1704926407, 'editable': 0, 'opt': 2, 'filter': 7, 'mentioned': [], 'is_hidden': False, 'can_follow_up': None, 'follow_up': None, 'submit_time': 1700535964, 'author_username': 'h*****t', 'author_portrait': '', 'author_shopid': 330025854, 'anonymous': True, 'images': ['vn-11134103-7r98o-loc17m6vu9dj4c', 'vn-11134103-7r98o-loc17m6vx2if65'], 'videos': [], 'product_items': [{'itemid': 21993222972, 'shopid': 288286284, 'name': 'Apple iPhone 15 Pro Max 256GB', 'image': 'vn-11134207-7r98o-llt6eul6d7tr35', 'is_snapshot': 1, 'snapshotid': 1551599355

In [ ]:
# product/
def get_reviews(shop_id,item_id):
    url = "https://shopee.com.my/api/v2/item/get_item_detail"
    limit = 1
    offset = 0
    reviews = []
    
    session = requests.Session()
    session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})
    
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "offset": offset,
        "limit": limit,
        "filter": "0",
        "flag": "0",
        "sort": "0",
        "append": "0",
        "before_bundle": "",
        "language": "en",
    }
    keyword_search = 'iphone 15 pro max'
    url = "https://shopee.sg/api/v2/search_items/?by=relevancy&keyword={}&limit=100&newest=0&order=desc&page_type=search".format(keyword_search)
    response = session.get(url, params=params).json()
    print(response)
    
# can change "relevancy" to "latest": to sort by latest products instead

# Shopee API request
    #     if response["error"]:
    #         print(f"Error: {response['error']}")
    #         break
    #     elif response["data"]["ratings"]:
    #         print(offset)
    #         print (response["data"]["ratings"])
    
    #         for rating in response["data"]["ratings"]:
    #             model_name = rating.get('product_items')[0].get('model_name')
    #             rating['model_name'] = model_name
    #             reviews.append(rating)
    #             offset += limit
    
    #             time.sleep(random.randint(2,5))  
    #     else:
    #         break
    
    # df = pd.DataFrame(reviews)
    
    # print(f"Retrieved {len(reviews)} reviews.")
    # # extract 'skin suitability' and 'absorption' from 'comment' column
    # df[['skin_suitability', 'absorption']] = df['comment'].str.extract('Skin Suitability:(.*?)\\nAbsorption:(.*?)\\n', expand=True)
    
    # # select columns to keep in the final CSV file
    # df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'rating', 'userid', 'shopid', 'comment', 'rating_star', 'skin_suitability', 'absorption', 'model_name']]
    # # df = df.to_string(index=False)
    
    # print('success')
    
    # # save the DataFrame as a CSV file
    # df.to_csv('ShopeeScrap1.csv', index=False)
shop_id = "288286284"
item_id = "21993222972"
get_reviews(shop_id,item_id)

{'error': 'error_not_found'}
